In [1]:
# Initialise environment
import pandas as pd
import matplotlib.pyplot as plt

RANDOM_SEED = 42

# Import data
data = pd.read_csv("C2-Grain-Only.csv")


In [2]:
data.describe()

,ET,NEE,ER,GPP,TMAX,TMIN,TAVG,HAVG,HDEG,CDEG,WCMN,WSPD,ATOT,RAIN,EVI
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,1.505212,0.543085,2.451847,1.908762,61.615876,35.239072,47.945464,69.911546,16.954536,0.381959,-100.518247,-10.025052,2.663299,0.048763,0.190907
std,0.847190,1.813217,1.710755,2.561796,13.432023,11.462908,11.468764,15.138623,10.941670,1.810977,338.352161,143.870108,102.692334,0.118245,0.145412
min,0.207102,-9.900786,0.610494,0.068792,32.050000,12.490000,24.220000,38.530000,0.000000,0.000000,-996.000000,-996.000000,-996.000000,0.000000,0.100000
25%,0.980321,0.088979,1.507960,0.715783,54.050000,27.150000,40.000000,57.640000,6.910000,0.000000,14.950000,7.500000,8.970000,0.000000,0.117600
50%,1.350401,0.721656,1.909361,1.113819,63.030000,32.670000,47.390000,70.870000,17.290000,0.000000,23.180000,9.470000,12.060000,0.000000,0.122400
75%,1.900066,1.332209,2.517847,1.811946,71.650000,43.220000,56.720000,80.360000,24.820000,0.000000,30.150000,13.040000,15.940000,0.010000,0.225000
max,4.902433,5.194033,8.067225,16.352418,90.360000,67.820000,78.570000,99.060000,40.780000,14.090000,48.980000,23.840000,29.670000,0.600000,0.983000


In [3]:
data.max()

ET       4.902433
NEE      5.194033
ER       8.067225
GPP     16.352418
TMAX    90.360000
TMIN    67.820000
TAVG    78.570000
HAVG    99.060000
HDEG    40.780000
CDEG    14.090000
WCMN    48.980000
WSPD    23.840000
ATOT    29.670000
RAIN     0.600000
EVI      0.983000
dtype: float64